# LR & DA

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from main import CIFAR10_dataset, CIFAR10_trainer
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm
import multiprocessing
import torch.optim as optim
import torch.nn.functional as  F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import v2
from torch.utils.data import default_collate
import random

In [40]:
def add_gaussian_noise(img, mean=0, std=0.1):
    noise = torch.randn(img.size()) * std + mean
    return img + noise

# Transformaciones para entrenamiento
train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomRotation(degrees=15),
    transforms.RandomGrayscale(p=0.1),
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
    
    transforms.ToTensor(),
    # transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3)),
    transforms.Lambda(lambda x: add_gaussian_noise(x, 0, 0.1)),
])
train_dataset = CIFAR10_dataset(partition="train", transform=train_transforms)
test_dataset = CIFAR10_dataset(partition="test")

# CutMix and MixUp
cutmix = v2.CutMix(num_classes=10)
mixup = v2.MixUp(num_classes=10)
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

def collate_fn(batch):
    data = default_collate(batch)  # Asegura el formato (inputs, targets)
    if random.random() < 0.4:
        return data
    inputs, labels= cutmix_or_mixup(data['img'], data['label']) # Aplica CutMix o MixUp
    return {"img": inputs, "label": labels}

####################################################################
# DataLoader Class
####################################################################

batch_size = 150
num_workers = multiprocessing.cpu_count()-1
print("Num workers", num_workers)
train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=num_workers, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size, shuffle=False, num_workers=num_workers)


Loading CIFAR10  train  Dataset...
Files already downloaded and verified
	Total Len.:  50000 
 --------------------------------------------------

Loading CIFAR10  test  Dataset...
Files already downloaded and verified
	Total Len.:  10000 
 --------------------------------------------------
Num workers 11


In [41]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride =1):
        super(BasicBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        # Shortcut
        self.conv_shortcut = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
        
        self.relu = nn.ReLU()

    def forward(self, x):
        previous_x = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        previous_x = self.conv_shortcut(previous_x)

        out += previous_x
        out = self.relu(out)

        return out


class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers
        self.layer1 = self._make_layer(32, 64, blocks=2, stride=2)   # conv2_x
        self.layer2 = self._make_layer(64, 128, blocks=2, stride=2)  # conv3_x
        self.layer3 = self._make_layer(128, 256, blocks=2, stride=2)  # conv4_x
        self.layer4 = self._make_layer(256, 512, blocks=1, stride=2)  # conv5_x

        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def _make_layer(self, in_channels, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(in_channels, out_channels, stride))
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)

        x = torch.flatten(x, 1)
        
        x = self.fc(x)
        x = self.dropout(x)
        return x

In [42]:
# Instantiating the network and printing its architecture
num_classes = 10
net = ResNet(num_classes)
print(net)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Params: ", count_parameters(net))

####################################################################
# Training settings
####################################################################

# Training hyperparameters
epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.02, weight_decay=1e-6)
lr_scheduler = optim.lr_scheduler.OneCycleLR(
    optimizer, max_lr=0.02, steps_per_epoch=len(train_dataloader), epochs=epochs
)


ResNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv_shortcut): Conv2d(32, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (relu): ReLU()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

In [43]:
trainer = CIFAR10_trainer(net, train_dataloader, test_dataloader, optimizer,criterion, epochs, lr_scheduler, batch_size=batch_size, early_stopping=25)

In [44]:
trainer.train()


---- Start Training ----


Test 0: 100%|██████████| 67/67 [00:01<00:00, 34.04batch/s]
/home/diego/.cache/pypoetry/virtualenvs/mlps-Z27HhEAS-py3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


[Epoch 1] Train Loss: 2.058454 - Test Loss: 1.737791 - Train Error: 72.45% - Test Error: 61.21%


Test 1: 100%|██████████| 67/67 [00:02<00:00, 33.45batch/s]


[Epoch 2] Train Loss: 1.912003 - Test Loss: 1.561755 - Train Error: 66.25% - Test Error: 52.47%


...

[Epoch 98] Train Loss: 1.366943 - Test Loss: 0.935591 - Train Error: 48.87% - Test Error: 25.96%



Test 98: 100%|██████████| 67/67 [00:02<00:00, 31.94batch/s]

[Epoch 99] Train Loss: 1.364951 - Test Loss: 0.980893 - Train Error: 48.47% - Test Error: 26.28%



Test 99: 100%|██████████| 67/67 [00:02<00:00, 32.02batch/s]

[Epoch 100] Train Loss: 1.382845 - Test Loss: 0.949055 - Train Error: 49.23% - Test Error: 26.04%

BEST TEST ERROR:  25.13  in epoch  87
